# Sistema de Recomendación

En este notebook vamos a crear un sistema de recomendación. Para ello seguiremos los siguientes pasos:
* Crear una tabla donde los indices sean los id de los juegos y haya 401 columnas con one-hot encoding representando "genres", "tags" y "specs".
* Utilizaremos esta tabla para crear una donde tanto filas como columnas sean id de juegos y los campos sean las distancias de coseno entre ellos.
* A partir de lo mismo armaremos un archivo json, donde las llaves serán los id de los juegos y los valores, una lista con los 5 id de juego mas similares.
* También crearemos un csv que contenga nombre e id de cada juego.
* Estos 2 archivos, json y csv, serán consumidos por la API para dar respuesta a su consulta sobre los 5 juegos mas similares al de referencia, y por tanto recomendados.

In [18]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# Cargar el archivo "steam_games_procesado.csv"
df = pd.read_csv("./Datasets/steam_games_procesado.csv")

# Eliminar las columnas "genres", "tags", "specs" y "year".
df.drop(columns=["genres", "tags", "specs", "year"], inplace=True)

df.head(2)

,id,app_name,genres_Accounting,genres_Action,genres_Adventure,genres_Animation & Modeling,genres_Audio Production,genres_Casual,genres_Design & Illustration,genres_Early Access,...,specs_Steam Achievements,specs_Steam Cloud,specs_Steam Leaderboards,specs_Steam Trading Cards,specs_Steam Turn Notifications,specs_Steam Workshop,specs_SteamVR Collectibles,specs_Tracked Motion Controllers,specs_Valve Anti-Cheat enabled,specs_Windows Mixed Reality
0,761140,Lost Summoner Kitty,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,643980,Ironbound,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


Primero haremos un dataframe para guardar la relación id/app_name y lo guardaremos en un .csv

In [20]:
# Crear el dataframe solo con las columnas id y app_name.
id_name_df = df[["id", "app_name"]]

# Guardar el dataframe con csv.
id_name_df.to_csv("./Datasets_API/id_name.csv", index=False)

Ahora en el dataframe original eliminaremos "app_name" y setearemos el indice en "id".

In [21]:
df.drop(columns=["app_name"], inplace=True)
df.set_index("id", inplace=True)
df.head(2)

,genres_Accounting,genres_Action,genres_Adventure,genres_Animation & Modeling,genres_Audio Production,genres_Casual,genres_Design & Illustration,genres_Early Access,genres_Education,genres_Free to Play,...,specs_Steam Achievements,specs_Steam Cloud,specs_Steam Leaderboards,specs_Steam Trading Cards,specs_Steam Turn Notifications,specs_Steam Workshop,specs_SteamVR Collectibles,specs_Tracked Motion Controllers,specs_Valve Anti-Cheat enabled,specs_Windows Mixed Reality
id,,,,,,,,,,,,,,,,,,,,,
761140,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
643980,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0


Vamos a calcular las similitudes de coseno entre los juegos y crear un dataframe que las contenga.

In [22]:
# Calcular la similitud de coseno entre las filas del DataFrame original.
cosine_sim = cosine_similarity(df.values)

# Crear un nuevo DataFrame con juegos como filas y columnas.
game_similarity_df = pd.DataFrame(cosine_sim, index=df.index, columns=df.index)

game_similarity_df.head()

id,761140,643980,670290,767400,773570,772540,774276,774277,774278,768800,...,771810,767590,747320,769330,745400,773640,733530,610660,658870,681550
id,,,,,,,,,,,,,,,,,,,,,
761140,1.000000,0.275241,0.527645,0.569803,0.381385,0.418121,0.345857,0.345857,0.345857,0.636364,...,0.533002,0.502519,0.301511,0.674200,0.609272,0.858116,0.636364,0.502519,0.454545,0.322329
643980,0.275241,1.000000,0.365148,0.069007,0.115470,0.101274,0.460739,0.460739,0.460739,0.220193,...,0.193649,0.243432,0.365148,0.244949,0.210819,0.346410,0.330289,0.304290,0.220193,0.097590
670290,0.527645,0.365148,1.000000,0.283473,0.316228,0.208013,0.688247,0.688247,0.688247,0.603023,...,0.265165,0.416667,0.250000,0.559017,0.360844,0.553399,0.452267,0.416667,0.376889,0.267261
767400,0.569803,0.069007,0.283473,1.000000,0.358569,0.524142,0.086711,0.086711,0.086711,0.341882,...,0.668153,0.377964,0.125988,0.507093,0.763763,0.358569,0.341882,0.125988,0.341882,0.303046
773570,0.381385,0.115470,0.316228,0.358569,1.000000,0.175412,0.290191,0.290191,0.290191,0.286039,...,0.335410,0.421637,0.210819,0.424264,0.365148,0.300000,0.286039,0.210819,0.286039,0.591608


Ahora crearemos un diccionario, donde la llave sea el id del juego y el valor, una lista con los 5 juegos con menor distancia de coseno (exculyendo al propio juego).

In [23]:
# Crear un diccionario para almacenar los resultados.
result_dict = {}

# Iterar a través de las filas del DataFrame
for index, row in game_similarity_df.iterrows():
    # Encontrar del 2º al 6º valor más alto en la fila actual.
    top_values = row.nlargest(6).index[1:6].tolist()
    
    # Almacenar el resultado en el diccionario con el índice de la fila como clave.
    result_dict[index] = top_values

Veamos los primeros elementos del diccionario para visualizar su estructura.

In [24]:
from itertools import islice

dict(islice(result_dict.items(), 3))

{761140: [693880, 699590, 705420, 705090, 705400],
 643980: [685420, 291410, 392620, 452230, 330000],
 670290: [522660, 729580, 590290, 427040, 701460]}

Ahora guardaremos este diccionario como un archivo json para ser consultado por la API.

In [25]:
with open('./Datasets_API/top_5_recomendados.json', 'w') as archivo_json:
    json.dump(result_dict, archivo_json)

Vamos a probar la función de la API.

In [26]:
def recomendacion_juego(id_ref: int):
    # Crear un dataframe a partir "top_5_recomendados.json".
    # Las columnas son los id del juego de referencia y las 5 filas los id
    # de los juegos mas similares.
    df = pd.read_json("./Datasets_API/top_5_recomendados.json")

    # Crear un dataframe a partir "id_name.csv".
    id_name_df = pd.read_csv("./Datasets_API/id_name.csv")

    res = {} # Crear un diccionario vacio para ser retornado como json.

    # df[id_ref] es una serie de Pandas correspondiente a la columna "id_ref"
    # del dataframe df. "val" contiene los id de los juegos
    for idx, val in df[id_ref].items():
        # Se obtiene el nombre del juego.
        name = id_name_df[id_name_df.id == val].app_name.item()
        # Se guarda en el diccionario el id como llave y el nombre como valor.
        res[val] = name

    return res

Cargaremos de nuevo "steam_games_procesado.csv" para probar esta función.

In [31]:
games_df = pd.read_csv("./Datasets/steam_games_procesado.csv")
games_df = games_df.iloc[:, :5]
games_df.head(3)

,id,app_name,genres,tags,specs
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player']
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']","['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult..."
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio...","['Free to Play', 'Simulation', 'Sports', 'Casu...","['Single-player', 'Multi-player', 'Online Mult..."


Probemos la función con el id 670290, correspondiente a el juego de Pool "Real Pool 3D - Poolians".

In [33]:
# Obtener las llaves (id) de los juegos recomendados
ids_recomendados = recomendacion_juego(670290).keys()

ids_recomendados

dict_keys([522660, 729580, 590290, 427040, 701460])

In [34]:
# Visualizar los juegos recomendados.
games_df[games_df.id.isin(ids_recomendados)]

,id,app_name,genres,tags,specs
7260,522660,Snooker-online multiplayer snooker game!,"['Casual', 'Free to Play', 'Indie', 'Simulatio...","['Free to Play', 'Sports', 'Casual', 'Simulati...","['Multi-player', 'Online Multi-Player', 'Cross..."
14556,729580,Malzbie's Pinball Collection,"['Casual', 'Free to Play', 'Indie', 'Simulatio...","['Free to Play', 'Indie', 'Casual', 'Simulatio...","['Single-player', 'Steam Achievements', 'Full ..."
15866,701460,Billiards,"['Casual', 'Indie', 'Simulation', 'Sports']","['Indie', 'Casual', 'Simulation', 'Sports']","['Single-player', 'Multi-player', 'Online Mult..."
19158,590290,Canvas The Gallery - Artist Pack,"['Casual', 'Free to Play', 'Indie', 'Simulation']","['Indie', 'Casual', 'Simulation', 'Free to Play']","['Single-player', 'Downloadable Content', 'In-..."
24256,427040,Pitstop Challenge,"['Casual', 'Indie', 'Simulation', 'Sports']","['Sports', 'Simulation', 'Indie', 'Casual']",['Single-player']


Podemos ver que los juegos son similares en género, etiquetas y especificaciones.